In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Import necessary libraries
import sys
from pathlib import Path
# Ensure the src directory is accessible
sys.path.append('./src')

# Download data

In [6]:
from get_data import get_data
data_split, ext_data = get_data()

Found local copy...
Loading...
Done!
100%|██████████| 13445/13445 [00:05<00:00, 2286.37it/s]

Data loaded and split successfully.
Train data samples: 9411
Validation data samples: 1344
Test data samples: 2690


# check data

In [7]:
data_split['train'].head(10)

,Drug_ID,Drug,Y
0,8640,O=C1NCCN1CC[N+]1CCC(c2cn(C3CCCCC3)c3ccc(Cl)cc2...,0
1,11377,O=C(Cc1ccc(-n2cnnn2)cc1)N1CCN(CCc2ccc3nonc3c2)CC1,1
2,1461,NC(=O)c1ncc(N[C@@H]2CCCC[C@@H]2N)cc1Nc1cccc(C(...,1
3,6646,Cc1cc(C)nc(Nc2cc(N[C@@H]3CCCC[C@@H]3N)cnc2C(N)...,1
4,379,COc1cc(C)nc(Nc2cc(N[C@@H]3CCCC[C@@H]3N)cnc2C(N...,1
5,10556,Cc1cc(CC(C)(C)O)cc(Nc2cc(N[C@@H]3CCCC[C@@H]3N)...,1
6,4542,Cc1cc(C(C)(C)O)cc(Nc2cc(N[C@@H]3CCCC[C@@H]3N)c...,1
7,9102,Cc1cccnc1CN1CCC2(CC1)C(=O)N(c1ccc(-c3ccc4scnc4...,1
8,4564,O=S1(=O)CCC(COc2ccc3c(c2)CCC2(CCN(C4CCC4)CC2)O...,0
9,12933,N#C[C@@H]1C[C@@H]2C[C@@H]2N1C(=O)[C@@H](N)C12C...,0


In [8]:
data_split['test'].head(10)

,Drug_ID,Drug,Y
0,7417,CC(C)c1noc(-c2nnc3n2CCN(C(=O)c2ccc(F)cc2)[C@@H...,1
1,3967,CCc1noc(-c2nnc3n2CCN(C(=O)c2ccc(F)cc2)[C@@H]3C)n1,0
2,13387,CC(C)(C)Cn1c(N)nc2ccc(-c3nc(C(C)(C)C)[nH]c3-c3...,0
3,10258,CC(C)(C)C1CCC2(CC1)CCN(c1ccc(OC(F)(F)F)cc1)C(=...,1
4,5612,CCN(CC)c1ccc2cc(C(=O)NCCCCN3CCC(Nc4nc5ccccc5n4...,1
5,6520,CN1CCC(COCc2cc(C(F)(F)F)cc(N3CCCC3)n2)(c2ccccc...,1
6,8954,CN1CCC(COCc2cc(C(F)(F)F)cc(N3CCC(C#N)C3)n2)(c2...,1
7,2983,FC(F)(F)c1cc(COCC2(c3ccccc3)CCNCC2)nc(N2CCCC2)c1,1
8,12756,OC1CCC(Nc2ncc3nc(Nc4c(F)cc(F)cc4F)n([C@@H]4CCO...,0
9,8043,O=C(N[C@@H]1C2CCN(CC2)[C@H]1Cc1cccnc1)c1cc2ccc...,0


In [10]:
ext_data['pos'].head(10)

,Y,Drug
0,0,CCOC(=O)C1(CCN(C)CC1)c1ccccc1
1,0,CCN(CC)CC(=O)NC1=C(C)C=CC=C1C
2,0,CCCC(CCC)C(=O)O
3,0,CCC(COC(=O)c1cc(OC)c(OC)c(OC)c1)(c1ccccc1)N(C)C
4,0,COc1ccc(N(C(C)=O)c2cc3c(cc2[N+](=O)[O-])OC(C)(...
5,1,COc1cc2nc(nc(N)c2cc1OC)N1CCN(CC1)C(=O)c1ccco1
6,1,COc1ccc2cc3-c4cc5OCOc5cc4CC[n+]3cc2c1OC
7,0,CCCc1nn(C)c2c(=O)[nH]c(nc12)c3cc(ccc3OCC)S(=O)...
8,0,CCCc1nn(C)c2c(=O)nc(-c3cc(S(=O)(=O)N4CCN(C)CC4...
9,0,CC(=O)SC1CC2=CC(=O)CCC2(C)C2CCC3C(CCC34CCC(=O)...


# check external data

In [12]:
# remove same smiles in the training
for i, curr_data in ext_data.items():
    for drug in curr_data['Drug']:
        if drug in data_split['train']['Drug']:
            print(drug)

In [14]:
for i, curr_data in ext_data.items():
    print(f"{i}: total ({len(curr_data)}); Y=1 ({curr_data['Y'].sum()})")
    

pos: total (44); Y=1 (30)
neg: total (41); Y=1 (11)
new: total (740); Y=1 (34)


# Featuring functions

In [15]:
from rdkit import Chem
from rdkit.Chem import AllChem

# Function to convert SMILES to Morgan fingerprints
def smiles_to_fp(smiles, radius=2, nBits=1024):
    mol = Chem.MolFromSmiles(smiles)
    return list(AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits))


# Prepare the data
def prepare_data(df):
    df['features'] = df['Drug'].apply(lambda x: smiles_to_fp(x))
    X = list(df['features'])
    y = df['Y'].values
    return X, y


# Prepare traing and test data

In [16]:

# Load data
train_data = data_split['train']
X_train, y_train = prepare_data(train_data)

test_data = data_split['test'] 
X_test, y_test = prepare_data(test_data)

# Random Forest Classifier

In [17]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train the model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)



RandomForestClassifier(random_state=42)

# Evaluation on internal test data

In [18]:
from evaluate import eval

# Evaluate the model
y_pred = rf_model.predict(X_test)
eval(y_test, y_pred, "RandomForest")

Accuracy: 0.7869888475836431
ROC AUC Score: 0.7866272534819189
Precision: 0.7987321711568938
Recall: 0.7596081386586285
F1 Score: 0.7786790266512167
Matthews Correlation Coefficient: 0.5742977119801898
Specificity (Negative Prediction Accuracy): 0.8136463683052091


# Evaluation on external data

In [20]:
for i, curr_data in ext_data.items():
    curr_X, curr_y = prepare_data(curr_data)
    curr_pred = rf_model.predict(curr_X)
    print(f"\n= = = Experiment on {i} = = =")
    eval(curr_y, curr_pred)



= = = Experiment on pos = = =
Accuracy: 0.8863636363636364
ROC AUC Score: 0.8976190476190476
Precision: 0.9629629629629629
Recall: 0.8666666666666667
F1 Score: 0.912280701754386
Matthews Correlation Coefficient: 0.7607036127908976
Specificity (Negative Prediction Accuracy): 0.9285714285714286

= = = Experiment on neg = = =
Accuracy: 0.9512195121951219
ROC AUC Score: 0.9378787878787879
Precision: 0.9090909090909091
Recall: 0.9090909090909091
F1 Score: 0.9090909090909091
Matthews Correlation Coefficient: 0.8757575757575757
Specificity (Negative Prediction Accuracy): 0.9666666666666667

= = = Experiment on new = = =
Accuracy: 0.9689189189189189
ROC AUC Score: 0.9417180469921678
Precision: 0.6078431372549019
Recall: 0.9117647058823529
F1 Score: 0.7294117647058823
Matthews Correlation Coefficient: 0.7301670806408677
Specificity (Negative Prediction Accuracy): 0.9716713881019831


# Simple Neural Network model

In [21]:
import torch

X_train = torch.tensor(X_train, dtype=torch.float32)  # Convert features to a float Tensor
y_train = torch.tensor(y_train, dtype=torch.float32)  # Convert labels to a float Tensor

# Ensure labels y_train is the right shape (e.g., for BCELoss, you might need to ensure it's two-dimensional if there are two outputs)
y_train = y_train.unsqueeze(1)  # Only do this if necessary

 # SimpleNN: Define the model

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(1024, 64),  # Assuming input features size of 1024
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),  # binary classification
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)


# SimpleNN: Define loss

In [23]:

def nn_loss_func(outputs, targets):
    criterion = nn.BCELoss()
    classical_loss = criterion(outputs.squeeze(), targets.squeeze())
    
    return classical_loss 

# SimpleNN: Training

In [24]:
# Assuming X_train and y_train are your datasets loaded as Tensor objects
nn_model = SimpleNN()
nn_optimizer = optim.Adam(nn_model.parameters(), lr=0.001)


# Training loop
epoch_n = 200
nn_model.train()
for epoch in range(epoch_n):
    nn_optimizer.zero_grad()
    outputs = nn_model(X_train)  # Make sure X_train is a tensor
    loss = nn_loss_func(outputs, y_train)  # Ensure y_train is appropriately shaped
    loss.backward()
    nn_optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")
    

Epoch 1, Loss: 0.6941598653793335
Epoch 2, Loss: 0.6913716793060303
Epoch 3, Loss: 0.6886349320411682
Epoch 4, Loss: 0.6856218576431274
Epoch 5, Loss: 0.6821855306625366
Epoch 6, Loss: 0.6782341599464417
Epoch 7, Loss: 0.6736916303634644
Epoch 8, Loss: 0.6685101985931396
Epoch 9, Loss: 0.6627156734466553
Epoch 10, Loss: 0.6563175320625305
Epoch 11, Loss: 0.6493382453918457
Epoch 12, Loss: 0.641829252243042
Epoch 13, Loss: 0.6338403224945068
Epoch 14, Loss: 0.625450849533081
Epoch 15, Loss: 0.6167259812355042
Epoch 16, Loss: 0.607739269733429
Epoch 17, Loss: 0.5985703468322754
Epoch 18, Loss: 0.589328408241272
Epoch 19, Loss: 0.5801218748092651
Epoch 20, Loss: 0.5710580348968506
Epoch 21, Loss: 0.5622432231903076
Epoch 22, Loss: 0.5537464618682861
Epoch 23, Loss: 0.5456221699714661
Epoch 24, Loss: 0.537885844707489
Epoch 25, Loss: 0.5305383205413818
Epoch 26, Loss: 0.523552656173706
Epoch 27, Loss: 0.5168911814689636
Epoch 28, Loss: 0.5105001330375671
Epoch 29, Loss: 0.5043290853500366


# SimpleNN: Internal Testing

In [25]:
# Assuming X_test and y_test are already loaded and are numpy arrays or lists
X_test = torch.tensor(X_test, dtype=torch.float32)  # Convert to tensor
# Ensure y_test is also a tensor if you have it and will evaluate metrics
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)  # Convert to tensor and ensure correct shape


In [26]:
# Set model to evaluation mode
nn_model.eval()

# Disable gradient computation for testing (saves memory and computations)
with torch.no_grad():
    outputs = nn_model(X_test)


# Convert outputs to predicted classes
# For binary classification with a single output unit
predictions = (outputs > 0.5).float()  # Threshold probabilities to classify as 1 or 0

# Since you cannot use sklearn directly with tensors, you need to move data back to CPU and convert to numpy
predictions = predictions.cpu().numpy()
y_test = y_test.cpu().numpy()

eval(y_test, predictions)

Accuracy: 0.7680297397769517
ROC AUC Score: 0.7679483784218618
Precision: 0.7664897649734648
Recall: 0.7618688771665411
F1 Score: 0.7641723356009069
Matthews Correlation Coefficient: 0.5359489111788915
Specificity (Negative Prediction Accuracy): 0.7740278796771827


# SimpleNN: evaluation on external data

In [29]:
for i, curr_data in ext_data.items():
    curr_X, curr_y = prepare_data(curr_data)    
    curr_X = torch.tensor(curr_X, dtype=torch.float32)  # Convert to tensor
    curr_y = torch.tensor(curr_y, dtype=torch.float32).unsqueeze(1)  # Convert to tensor and ensure correct shape
    
    # Set model to evaluation mode
    nn_model.eval()

    # Disable gradient computation for testing (saves memory and computations)
    with torch.no_grad():
        outputs = nn_model(curr_X)
    
    curr_pred = (outputs > 0.5).float()  # Threshold probabilities to classify as 1 or 0

    # Since you cannot use sklearn directly with tensors, you need to move data back to CPU and convert to numpy
    curr_pred = curr_pred.cpu().numpy()
    curr_y = curr_y.cpu().numpy()

    print(f"\n= = = Experiment on {i} = = =")
    eval(curr_y, curr_pred)



= = = Experiment on pos = = =
Accuracy: 0.8636363636363636
ROC AUC Score: 0.880952380952381
Precision: 0.9615384615384616
Recall: 0.8333333333333334
F1 Score: 0.8928571428571429
Matthews Correlation Coefficient: 0.7217759492810224
Specificity (Negative Prediction Accuracy): 0.9285714285714286

= = = Experiment on neg = = =
Accuracy: 0.8048780487804879
ROC AUC Score: 0.8090909090909092
Precision: 0.6
Recall: 0.8181818181818182
F1 Score: 0.6923076923076923
Matthews Correlation Coefficient: 0.5686445668048217
Specificity (Negative Prediction Accuracy): 0.8

= = = Experiment on new = = =
Accuracy: 0.9445945945945946
ROC AUC Score: 0.9289701716380603
Precision: 0.4492753623188406
Recall: 0.9117647058823529
F1 Score: 0.6019417475728155
Matthews Correlation Coefficient: 0.6177504833839682
Specificity (Negative Prediction Accuracy): 0.9461756373937678


# LTN first trial

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class LTNModel(nn.Module):
    def __init__(self):
        super(LTNModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(1024, 64),  # Assuming input features size of 1024
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),  # binary classification
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)
    
    
    
# Instantiate the model
model = LTNModel()

# Prepare data
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Setup optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define and run the training function
def train(model, X_train, y_train, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)  # Directly use the model
        loss = nn.BCELoss()(outputs, y_train)
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# Execute the training
train(model, X_train_tensor, y_train_tensor, optimizer)


# Set model to evaluation mode
nn_model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
predictions = (outputs > 0.5).float()  # Threshold probabilities to classify as 1 or 0

# Since you cannot use sklearn directly with tensors, you need to move data back to CPU and convert to numpy
predictions = predictions.cpu().numpy()
y_test_tensor = y_test_tensor.cpu().numpy()
eval(y_test_tensor, predictions)

# Logic Tensor Network (LTN)

In [ ]:
import ltn

In [ ]:
class LTNModel(torch.nn.Module):
    def __init__(self):
        super(LTNModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(1024, 64),  # Assuming input features size of 1024
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),  # binary classification
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)
        
    
# Instantiate model and predicate
ltn_model = LTNModel()
#predicate_toxic = ltn.Predicate(ltn_model)


# LTN: Rules

In [ ]:
# Rule: Structural similarity implies similar toxicity
# Define a similarity measure (could be cosine similarity, Jaccard index, etc. for molecular fingerprints)
def similarity(x, y):
    return torch.cosine_similarity(x, y, dim=1)

# Logical rule
def rule_structural_similarity(data):
    for i in range(len(data)):
        for j in range(len(data)):
            # Enforce that similar compounds should have similar toxicity predictions
            ltn.axiom((similarity(data[i], data[j]) > 0.8) >> (ltn.equivalence(predicate_toxic(data[i]), predicate_toxic(data[j]))))


            
def structure_similarity(data, epsilon=0.85):
    # Assume a function that calculates pairwise similarity (e.g., cosine similarity)
    similarities = torch.mm(data, data.t())  # This is a simplistic way to compute similarities
    for i in range(len(data)):
        for j in range(len(data)):
            # Adding similarity rule: similar structures imply similar toxicity levels
            ltn.axiom((similarities[i, j] > epsilon) >> (ltn.equivalence(predicate_toxic(data[i]), predicate_toxic(data[j]))))

# Dummy function for presence of specific substructures (you need actual implementation based on your data)
def presence_of_toxic_substructures(data, indices_of_substructures):
    for i in range(len(data)):
        if any(data[i][idx] == 1 for idx in indices_of_substructures):
            ltn.axiom(predicate_toxic(data[i]))            


# Prepare Data

In [ ]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)


# LTN: Training

In [ ]:
ltn_optimizer = optim.Adam(ltn_model.parameters(), lr=0.001)

# Define a training function
def ltn_train(predicate, X_train, y_train, optimizer, epochs=100):
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = predicate(X_train)  # Using the predicate directly
        loss = nn.BCELoss()(outputs.tensor, y_train.tensor)  # Accessing the tensor from the LTN Tensor
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}, Loss: {loss.item()}")
            
# Step 7: Run the training
# Execute the training
ltn_train(predicate_toxic, X_train_ltn, y_train_ltn, ltn_optimizer)

In [ ]:

model.eval()
with torch.no_grad():
    test_outputs = predicate_toxic(X_test_term)
    test_predictions = (test_outputs > 0.5).float()
    accuracy = (test_predictions == y_test_term.tensor).float().mean()  # Compare against the tensor inside the term
    print(f"Test Accuracy: {accuracy.item()}")